Our task is to build a consultant bot that can answer questions of different domains, such as medical with a doctor bot or legal with a lawyer bot.

We will show how flexible ``Component`` and the ``Sequential`` container is to build the same task
in different ways.

1. **Single Task**: We can build a single task where it deals with multiple generators and handles any coding logic.
2. **Multiple Tasks** and combine them using ``Sequential`` which resembles the concept of `Chain` or pipelines in other libraries.

First, lets prepare the imports and prompt templates using `jinjia2` template. We plan to demonstrate how we can use different models too. If this tutorial is the first thing you read, no need to care about more details, but focus on how the `development process` looks like using `LightRAG` library.

In [26]:
import re
from lightrag.core import Component, Generator, Sequential
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [27]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""
template_law = r"""<SYS> You are a lawyer </SYS> User: {{input_str}}"""
template_router = r"""<SYS> You are a router who will route a user question to the right generator.
            Here are your choices in form of key: value pairs:
             {% for key, value in choices.items() %}
                {{ key }}: {{ value }}
             {% endfor %}
            Output the key of your choice.
            </SYS> User question: {{input_str}}
            You:
            """

In [28]:
# Let's turn on the library log to help with debugging.
from lightrag.utils import get_logger
get_logger()

Here is our first approach to build a single task with multiple generators and call each conditionally.

In [29]:
class ChatBotWithRouter(Component):
    def __init__(self):
        super().__init__()
        model_1_kwargs = {
            "model": "gpt-3.5-turbo",
        }
        model_2_kwargs = {"model": "llama3-8b-8192"}
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs=model_2_kwargs,
        )
        self.router_choices = {
            "doctor": self.create_generator_signature(self.doc),
            "lawyer": self.create_generator_signature(self.lawyer),
            "other": "Choose me the question does not apply to other choices.",
        }
        print(self.router_choices)

        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )

    def call(self, query: str) -> str:
        choice = self.router(
            prompt_kwargs={"input_str": query, "choices": self.router_choices}
        ).data
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Sorry, I cannot help you with that."

    def create_generator_signature(self, generator: Generator):
        template = generator.template
        pattern = r"<SYS>(.*?)</SYS>"

        matches = re.findall(pattern, template)
        for match in matches:
            print("Content between <SYS> tags:", match)
            return match

In [30]:
# Initiate the task component, and print the task details.

task = ChatBotWithRouter()
task

2024-06-09 23:13:16 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 23:13:16 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
Content between <SYS> tags:  You are a doctor 
Content between <SYS> tags:  You are a lawyer 
{'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}
2024-06-09 23:13:16 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']


ChatBotWithRouter(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
                  </SYS> 

In [31]:
# Call the task with a query

query = "I have a legal question"
print(task(query))

2024-06-09 23:13:16 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}}
2024-06-09 23:13:16 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-09 23:13:16 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor:  You are a doctor \n                lawyer:  You are a lawyer \n                other: Choose me the question does not apply to other choices.\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 23:13:17 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https:

Now, let's separate this into multiple subtasks and ``chain`` them using the ``Sequential`` container.

First, the router task which will takes a dictionary of choices and return the selected key. In addition, we use ``_extra_repr`` to improve the default string representation of the task.

As ``Sequential`` will pass the output of one task to the next using positional arguments, we return whatever is needed to the next task in a dictionary.

In [32]:
# Router component

from typing import Dict
class Router(Component):
    def __init__(self, choices: Dict[str, str] = {}):
        super().__init__()
        self.choices = choices
        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        prompt_kwargs = {"input_str": query, "choices": self.choices}
        choice =  self.router(prompt_kwargs=prompt_kwargs).data
        return {"choice": choice, "query": query}
    
    def _extra_repr(self):
        return f"Choices: {self.choices}, "

In [33]:
r = Router()
r

2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']


Router(
  Choices: {}, 
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
                  </SYS> User question: {{input_str}}
                  You:
                  , prompt_variables: ['input_str', 'choices']
    )
    (model_client): OpenAIClient()
  )
)

Now, lets build another subtask which handles the chat depending on the selected key from the router task.
As the router task returns a dictionary, we will make our input dictionary type that parses the ``choice`` and ``query`` key value pairs.

In [34]:
# the second chat component with two generators

class Chat(Component):
    def __init__(self):
        super().__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs={"model": "llama3-8b-8192"},
        )
    # to chain together just to make sure the output can be directly passed to the next as input
    def call(self, input: Dict[str, str]) -> Dict[str, str]:
        choice = input.get("choice", None)
        query = input.get("query", None)
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Sorry, I am not able to help you with that."

In [35]:
chat = Chat()
chat

2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


Chat(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
)

Now, lets chain the router and the chat task using the ``Sequential`` container into a runnable pipeline.

In [36]:
class QAWithRouter(Component):
    def __init__(self):
        super().__init__()
        self.router = Router(choices={"doctor": "Doctor", "lawyer": "Lawyer", "other": "Other"})
        self.chat = Chat()
        self.pipeline = Sequential(self.router, self.chat)

    def call(self, query: str) -> str:
        return self.pipeline(query)

In [37]:
qa = QAWithRouter()
qa

2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']
2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 23:13:17 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


QAWithRouter(
  (router): Router(
    Choices: {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}, 
    (router): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(
        template: <SYS> You are a router who will route a user question to the right generator.
                    Here are your choices in form of key: value pairs:
                     {% for key, value in choices.items() %}
                        {{ key }}: {{ value }}
                     {% endfor %}
                    Output the key of your choice.
                    </SYS> User question: {{input_str}}
                    You:
                    , prompt_variables: ['input_str', 'choices']
      )
      (model_client): OpenAIClient()
    )
  )
  (chat): Chat(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables

In [38]:
qa("I have a legal question")

2024-06-09 23:13:18 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}}
2024-06-09 23:13:18 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-09 23:13:18 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor: Doctor\n                lawyer: Lawyer\n                other: Other\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 23:13:18 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 23:13:18 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='lawyer',

"I'd be happy to help you with your legal question! Can you please provide some more details about your question, such as what type of law you're dealing with (e.g. criminal, family, employment, etc.) and what specific issues or concerns you're facing?"